In [1]:
import numpy as np
import pandas as pd


# 数据包含两个类别 man 和 woman，特征分别为：hair（头发长短），voice（声音粗细），height（身高），ear_stud（是否带有耳钉）
def create_data():
    # 生成示例数据
    data_value = np.array(
        [
            ["long", "thick", 175, "no", "man"],
            ["short", "medium", 168, "no", "man"],
            ["short", "thin", 178, "yes", "man"],
            ["short", "thick", 172, "no", "man"],
            ["long", "medium", 163, "no", "man"],
            ["short", "thick", 180, "no", "man"],
            ["long", "thick", 173, "yes", "man"],
            ["short", "thin", 174, "no", "man"],
            ["long", "thin", 164, "yes", "woman"],
            ["long", "medium", 158, "yes", "woman"],
            ["long", "thick", 161, "yes", "woman"],
            ["short", "thin", 166, "yes", "woman"],
            ["long", "thin", 158, "no", "woman"],
            ["short", "medium", 163, "no", "woman"],
            ["long", "thick", 161, "yes", "woman"],
            ["long", "thin", 164, "no", "woman"],
            ["short", "medium", 172, "yes", "woman"],
        ]
    )
    columns = np.array(["hair", "voice", "height", "ear_stud", "labels"])
    data = pd.DataFrame(data_value.reshape(17, 5), columns=columns)
    return data

In [3]:
data = create_data()
data.head()

,hair,voice,height,ear_stud,labels
0,long,thick,175,no,man
1,short,medium,168,no,man
2,short,thin,178,yes,man
3,short,thick,172,no,man
4,long,medium,163,no,man


In [ ]:
# 以上数据只有 man 和 woman,man 占据 8 / 17, women 占 9 / 17

# 对应到信息熵公式则有
# Ent(data) = -(8 / 17 * log2(8 / 17) + 9 / 17 * log2(9 / 17)) = 0.9975

In [7]:
import math


def get_Ent(data):
    """
    参数:
    data -- 数据集

    返回:
    Ent -- 信息熵
    """
    # 计算信息熵
    num_sample = len(data)  # 样本个数
    label_counts = {}  # 初始化标签统计字典
    for i in range(num_sample):
        each_data = data.iloc[i, :]
        current_label = each_data["labels"]  # 得到当前元素的标签（label）

        # 如果标签不在当前字典中，添加该类标签并初始化 value=0,否则该类标签 value+1
        if current_label not in label_counts.keys():
            label_counts[current_label] = 0
        label_counts[current_label] += 1
    print(f"label_counts: {label_counts}")
    Ent = 0.0  # 初始化信息熵
    for key in label_counts:
        prob = float(label_counts[key]) / num_sample
        Ent -= prob * math.log(prob, 2)  # 应用信息熵公式计算信息熵
    return Ent

In [8]:
base_ent = get_Ent(data)
base_ent

label_counts: {'man': 8, 'woman': 9}


0.9975025463691153

In [9]:
def get_gain(data, base_ent, feature):
    """
    参数:
    data -- 数据集
    base_ent -- 根节点的信息熵
    feature -- 计算信息增益的特征

    返回:
    Ent -- 信息熵
    """
    # 计算信息增益
    feature_list = data[feature]  # 得到一个特征的全部取值
    unique_value = set(feature_list)  # 特征取值的类别
    feature_ent = 0.0

    for each_feature in unique_value:
        temp_data = data[data[feature] == each_feature]
        weight = len(temp_data) / len(feature_list)  # 计算该特征的权重值
        temp_ent = weight * get_Ent(temp_data)
        feature_ent = feature_ent + temp_ent

    gain = base_ent - feature_ent  # 信息增益
    return gain

In [10]:
get_gain(data, base_ent, "hair")

label_counts: {'man': 3, 'woman': 6}
label_counts: {'man': 5, 'woman': 3}


0.062200515199107964

In [19]:
# 通过二分法处理连续值
# 计算 连续值最优划分点
def get_splitpoint(data, base_ent, feature):
    """
    参数:
    data -- 数据集
    base_ent -- 根节点的信息熵
    feature -- 需要划分的连续特征

    返回:
    final_t -- 连续值最优划分点
    """
    # 将连续值进行排序并转化为浮点类型
    continues_value = data[feature].sort_values().astype(np.float64)
    continues_value = [i for i in continues_value]  # 不保留原来的索引
    t_set = []
    t_ent = {}

    # 得到划分点 t 的集合
    for i in range(len(continues_value) - 1):
        temp_t = (continues_value[i] + continues_value[i + 1]) / 2
        t_set.append(temp_t)
    print(f"continues_value: {continues_value} {len(continues_value)}")
    print(f"t_set: {t_set} {len(t_set)}")
    print()
    # 计算最优划分点
    for each_t in t_set:
        # 将大于划分点的分为一类
        temp1_data = data[data[feature].astype(np.float64) > each_t]
        # 将小于划分点的分为一类
        temp2_data = data[data[feature].astype(np.float64) < each_t]

        weight1 = len(temp1_data) / len(data)
        weight2 = len(temp2_data) / len(data)
        # 计算每个划分点的信息增益
        temp_ent = (
            base_ent - weight1 * get_Ent(temp1_data) - weight2 * get_Ent(temp2_data)
        )
        t_ent[each_t] = temp_ent
    print("t_ent:", t_ent)
    final_t = max(t_ent, key=t_ent.get)
    return final_t

In [20]:
final_t = get_splitpoint(data, base_ent, "height")
final_t

continues_value: [158.0, 158.0, 161.0, 161.0, 163.0, 163.0, 164.0, 164.0, 166.0, 168.0, 172.0, 172.0, 173.0, 174.0, 175.0, 178.0, 180.0] 17
t_set: [158.0, 159.5, 161.0, 162.0, 163.0, 163.5, 164.0, 165.0, 167.0, 170.0, 172.0, 172.5, 173.5, 174.5, 176.5, 179.0] 16

label_counts: {'man': 8, 'woman': 7}
label_counts: {}
label_counts: {'man': 8, 'woman': 7}
label_counts: {'woman': 2}
label_counts: {'man': 8, 'woman': 5}
label_counts: {'woman': 2}
label_counts: {'man': 8, 'woman': 5}
label_counts: {'woman': 4}
label_counts: {'man': 7, 'woman': 4}
label_counts: {'woman': 4}
label_counts: {'man': 7, 'woman': 4}
label_counts: {'man': 1, 'woman': 5}
label_counts: {'man': 7, 'woman': 2}
label_counts: {'man': 1, 'woman': 5}
label_counts: {'man': 7, 'woman': 2}
label_counts: {'man': 1, 'woman': 7}
label_counts: {'man': 7, 'woman': 1}
label_counts: {'man': 1, 'woman': 8}
label_counts: {'man': 6, 'woman': 1}
label_counts: {'man': 2, 'woman': 8}
label_counts: {'man': 5}
label_counts: {'man': 2, 'woman

172.0

In [21]:
# 利用找出的分界值，将数据分为两类
def choice_1(x, t):
    if x > t:
        return ">{}".format(t)
    else:
        return "<{}".format(t)


deal_data = data.copy()
# 使用lambda和map函数将 height 按照final_t划分为两个类别
deal_data["height"] = pd.Series(
    map(lambda x: choice_1(int(x), final_t), deal_data["height"])
)
deal_data

,hair,voice,height,ear_stud,labels
0,long,thick,>172.0,no,man
1,short,medium,<172.0,no,man
2,short,thin,>172.0,yes,man
3,short,thick,<172.0,no,man
4,long,medium,<172.0,no,man
5,short,thick,>172.0,no,man
6,long,thick,>172.0,yes,man
7,short,thin,>172.0,no,man
8,long,thin,<172.0,yes,woman
9,long,medium,<172.0,yes,woman


In [22]:
# 计算信息增益最大的特征
def choose_feature(data):
    """
    参数:
    data -- 数据集

    返回:
    best_feature -- 最优的划分特征
    """
    # 选择最优划分特征
    num_features = len(data.columns) - 1  # 特征数量
    base_ent = get_Ent(data)
    best_gain = 0.0  # 初始化信息增益
    best_feature = data.columns[0]
    for i in range(num_features):  # 遍历所有特征
        temp_gain = get_gain(data, base_ent, data.columns[i])  # 计算信息增益
        if temp_gain > best_gain:  # 选择最大的信息增益
            best_gain = temp_gain
            best_feature = data.columns[i]
    return best_feature  # 返回最优特征

In [23]:
choose_feature(deal_data)

label_counts: {'man': 8, 'woman': 9}
label_counts: {'man': 3, 'woman': 6}
label_counts: {'man': 5, 'woman': 3}
label_counts: {'man': 2, 'woman': 4}
label_counts: {'man': 4, 'woman': 2}
label_counts: {'man': 2, 'woman': 3}
label_counts: {'man': 3, 'woman': 9}
label_counts: {'man': 5}
label_counts: {'man': 6, 'woman': 3}
label_counts: {'man': 2, 'woman': 6}


'height'

In [24]:
# 反复遍历数据集，计算每一个特征的信息增益，通过比较将最好的特征作为父节点，根据特征的值确定分支子节点，然后重复以上操作，直到某一个分支全部属于同一类别，或者遍历完所有的数据特征，当遍历到最后一个特征时，若分支数据依然「不纯」，就将其中数量较多的类别作为子节点

def create_tree(data):
    """
    参数:
    data -- 数据集

    返回:
    tree -- 以字典的形式返回决策树
    """
    # 构建决策树
    feature_list = data.columns[:-1].tolist()
    label_list = data.iloc[:, -1]
    if len(data["labels"].value_counts()) == 1:
        leaf_node = data["labels"].mode().values
        return leaf_node  # 第一个递归结束条件：所有的类标签完全相同
    if len(feature_list) == 1:
        leaf_node = data["labels"].mode().values
        return leaf_node  # 第二个递归结束条件：用完了所有特征
    best_feature = choose_feature(data)  # 最优划分特征
    tree = {best_feature: {}}
    feat_values = data[best_feature]
    unique_value = set(feat_values)
    for value in unique_value:
        temp_data = data[data[best_feature] == value]
        temp_data = temp_data.drop([best_feature], axis=1)
        tree[best_feature][value] = create_tree(temp_data)
    return tree

In [25]:
tree = create_tree(deal_data)
tree

label_counts: {'man': 8, 'woman': 9}
label_counts: {'man': 3, 'woman': 6}
label_counts: {'man': 5, 'woman': 3}
label_counts: {'man': 2, 'woman': 4}
label_counts: {'man': 4, 'woman': 2}
label_counts: {'man': 2, 'woman': 3}
label_counts: {'man': 3, 'woman': 9}
label_counts: {'man': 5}
label_counts: {'man': 6, 'woman': 3}
label_counts: {'man': 2, 'woman': 6}
label_counts: {'man': 3, 'woman': 9}
label_counts: {'man': 2, 'woman': 3}
label_counts: {'man': 1, 'woman': 6}
label_counts: {'woman': 4}
label_counts: {'man': 1, 'woman': 2}
label_counts: {'man': 2, 'woman': 3}
label_counts: {'man': 3, 'woman': 3}
label_counts: {'woman': 6}
label_counts: {'man': 3, 'woman': 3}
label_counts: {'man': 2, 'woman': 1}
label_counts: {'man': 1, 'woman': 2}
label_counts: {'woman': 2}
label_counts: {'man': 1}
label_counts: {'man': 2, 'woman': 1}


{'height': {'<172.0': {'ear_stud': {'no': {'voice': {'thin': array(['woman'], dtype=object),
      'thick': array(['man'], dtype=object),
      'medium': array(['man'], dtype=object)}},
    'yes': array(['woman'], dtype=object)}},
  '>172.0': array(['man'], dtype=object)}}

In [32]:
def classify(tree, test):
    """
    参数:
    data -- 数据集
    test -- 需要测试的数据

    返回:
    class_label -- 分类结果
    """
    # 决策分类
    first_feature = list(tree.keys())[0]  # 获取根节点
    feature_dict = tree[first_feature]  # 根节点下的树
    labels = test.columns.tolist()
    value = test[first_feature][0]
    for key in feature_dict.keys():
        if value == key:
            if type(feature_dict[key]).__name__ == "dict":  # 判断该节点是否为叶节点
                class_label = classify(feature_dict[key], test)  # 采用递归直到遍历到叶节点
            else:
                class_label = feature_dict[key]
    return class_label

In [33]:
test = pd.DataFrame(
    {"hair": ["long"], "voice": ["thin"], "height": [163], "ear_stud": ["yes"]}
)
test

,hair,voice,height,ear_stud
0,long,thin,163,yes


In [34]:
test["height"] = pd.Series(map(lambda x: choice_1(int(x), final_t), test["height"]))
test

,hair,voice,height,ear_stud
0,long,thin,<172.0,yes


In [35]:
classify(tree, test)

array(['woman'], dtype=object)